## Google News data

Use the Google News API 

`72bc383e0bd44a5cb0997c468f7e0a2e`

In [0]:
import requests
import json
import codecs
import csv
import pandas as pd

In [0]:
file = codecs.open('movie.json', 'w', encoding='utf-8')
#API
API_KEY = '72bc383e0bd44a5cb0997c468f7e0a2e'
url = 'https://newsapi.org/v2/everything?q=bitcoin&apiKey='+API_KEY
#
start=5
count=25
r = requests.get(url, params={'start': start, 'count': count})
r.encoding='UTF_8'
content=r.json()
file.write(json.dumps(content, ensure_ascii=False))


In [85]:
csvfile = open('results_1.csv', 'w')
header = ['Name','Title','description','url','publishedAt']
# filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter = csv.writer(csvfile)
filewriter.writerow(header)

40

In [0]:
# with open('movie.json') as json_data:
#     data = json.load(json_data,) 
# 
# for item in data["articles"]:
#   print(item["source"]["name"])
#   print(item["title"])
#   print(item["description"])
#   print(item["url"])
#   print(item["publishedAt"])
#   print("\n")

In [0]:
with open('movie.json') as json_data:
    data = json.load(json_data,) 

for item in data["articles"]:
  temp = []
  temp.append(item["source"]["name"])
  temp.append(item["title"])
  temp.append(item["description"])
  temp.append(item["url"])
  temp.append(item["publishedAt"])
  filewriter.writerow(temp)
